Notebook to visualize results

In [1]:
import os
import itertools

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pingouin as pg
import scikit_posthocs as sp
import seaborn as sns
#from cliffs_delta import cliffs_delta
from scipy import stats



In [167]:
variables =['global_energy',
                "avg_cpu_usage_config",'avg_used_memory_pct_config',
                'avg_utilization_gpu_config','avg_utilization_memory_config','avg_used_memory_pct_mib',
                'avg_Load', 'duration'
                ]
dependent_variable = 'avg_utilization_gpu_config'

In [168]:
if dependent_variable in ['global_energy',]:
    cpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_cpu/tables/median_energy_values.csv", index_col=None, header=0)
    gpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu_test/tables/median_energy_values.csv", index_col=None, header=0)
elif dependent_variable in ["avg_cpu_usage_config",'avg_used_memory_pct_config',]:
    cpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_cpu/tables/median_resource_energi.csv", index_col=None, header=0)
    gpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu_test/tables/median_resource_energi.csv", index_col=None, header=0)
elif dependent_variable in ['avg_utilization_gpu_config','avg_utilization_memory_config','avg_used_memory_pct_mib',]:
    cpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_cpu/tables/median_resource_nvidia.csv", index_col=None, header=0)
    gpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu_test/tables/median_resource_nvidia.csv", index_col=None, header=0)
elif dependent_variable in ['avg_Load',] :
    cpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_cpu/tables/median_resource_wattmeter.csv", index_col=None, header=0)
    gpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu_test//tables/median_resource_wattmeter.csv", index_col=None, header=0)
elif dependent_variable in ['duration', ]:
    cpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_cpu/tables/median_time_inference.csv", index_col=None, header=0)
    gpu_ep_data = pd.read_csv("D:/GAISSA/energy-repo/last_repo/june_gpu_test/tables/median_time_inference.csv", index_col=None, header=0)

print(len(cpu_ep_data))
print(len(gpu_ep_data))

21
16


In [169]:
print(len(cpu_ep_data['label'].unique()))
print(cpu_ep_data['label'].unique())

print(len(gpu_ep_data['label'].unique()))
print(gpu_ep_data['label'].unique())


21
['idle_idle' 'onnx_codeparrot-small' 'onnx_phi2' 'onnx_pythia-410m'
 'onnx_pythia1-4b' 'onnx_tinyllama' 'ov_codeparrot-small' 'ov_phi2'
 'ov_pythia-410m' 'ov_pythia1-4b' 'ov_tinyllama' 'torch_codeparrot-small'
 'torch_phi2' 'torch_pythia-410m' 'torch_pythia1-4b' 'torch_tinyllama'
 'torchscript_codeparrot-small' 'torchscript_phi2'
 'torchscript_pythia-410m' 'torchscript_pythia1-4b'
 'torchscript_tinyllama']
16
['idle_idle' 'onnx_codeparrot-small' 'onnx_phi2' 'onnx_pythia-410m'
 'onnx_pythia1-4b' 'onnx_tinyllama' 'torch_codeparrot-small' 'torch_phi2'
 'torch_pythia-410m' 'torch_pythia1-4b' 'torch_tinyllama'
 'torchscript_codeparrot-small' 'torchscript_phi2'
 'torchscript_pythia-410m' 'torchscript_pythia1-4b'
 'torchscript_tinyllama']


In [170]:
print(cpu_ep_data.columns)
print(gpu_ep_data.columns)

assert 'label' in cpu_ep_data.columns
assert 'label' in gpu_ep_data.columns
assert dependent_variable in cpu_ep_data.columns
assert dependent_variable in gpu_ep_data.columns



Index(['runtime', 'model', 'label', 'avg_cpu_usage_config', 'avg_USED_MEMORY',
       'avg_used_memory_pct_config'],
      dtype='object')
Index(['runtime', 'model', 'label', 'avg_cpu_usage_config', 'avg_USED_MEMORY',
       'avg_used_memory_pct_config'],
      dtype='object')


In [171]:
cpu_ep_data

,runtime,model,label,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config
0,idle,idle,idle_idle,0.130797,2.501843e+09,3.757879
1,onnx,codeparrot-small,onnx_codeparrot-small,27.016334,9.507387e+09,14.280513
2,onnx,phi2,onnx_phi2,79.619104,1.985527e+10,29.823487
3,onnx,pythia-410m,onnx_pythia-410m,30.478362,1.173323e+10,17.623832
4,onnx,pythia1-4b,onnx_pythia1-4b,39.844363,9.955764e+09,14.953995
5,onnx,tinyllama,onnx_tinyllama,81.593727,1.000236e+10,15.023981
6,ov,codeparrot-small,ov_codeparrot-small,28.739977,1.167752e+10,17.540144
7,ov,phi2,ov_phi2,44.345280,1.285510e+10,19.308921
8,ov,pythia-410m,ov_pythia-410m,34.946671,1.179588e+10,17.717936
9,ov,pythia1-4b,ov_pythia1-4b,41.236916,1.309087e+10,19.663062


In [172]:
cpu_ep_data['config'] = "cpuep_"+cpu_ep_data['label']
gpu_ep_data['config'] = "gpuep_"+gpu_ep_data['label']

gpu_ep_data

,runtime,model,label,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config,config
0,idle,idle,idle_idle,0.110316,2.495251e+09,3.747977,gpuep_idle_idle
1,onnx,codeparrot-small,onnx_codeparrot-small,3.991848,5.463905e+09,8.207026,gpuep_onnx_codeparrot-small
2,onnx,phi2,onnx_phi2,3.418926,6.012355e+09,9.030823,gpuep_onnx_phi2
3,onnx,pythia-410m,onnx_pythia-410m,3.422415,6.463005e+09,9.707720,gpuep_onnx_pythia-410m
4,onnx,pythia1-4b,onnx_pythia1-4b,3.379086,4.971254e+09,7.467044,gpuep_onnx_pythia1-4b
5,onnx,tinyllama,onnx_tinyllama,3.736836,5.061456e+09,7.602531,gpuep_onnx_tinyllama
6,torch,codeparrot-small,torch_codeparrot-small,2.443924,3.546823e+09,5.327485,gpuep_torch_codeparrot-small
7,torch,phi2,torch_phi2,3.013264,8.007964e+09,12.028316,gpuep_torch_phi2
8,torch,pythia-410m,torch_pythia-410m,2.491036,3.558276e+09,5.344688,gpuep_torch_pythia-410m
9,torch,pythia1-4b,torch_pythia1-4b,2.993023,3.378596e+09,5.074801,gpuep_torch_pythia1-4b


In [173]:
global_df = pd.concat([cpu_ep_data,gpu_ep_data], axis=0, ignore_index=True)
print(global_df['config'].unique())
global_df

['cpuep_idle_idle' 'cpuep_onnx_codeparrot-small' 'cpuep_onnx_phi2'
 'cpuep_onnx_pythia-410m' 'cpuep_onnx_pythia1-4b' 'cpuep_onnx_tinyllama'
 'cpuep_ov_codeparrot-small' 'cpuep_ov_phi2' 'cpuep_ov_pythia-410m'
 'cpuep_ov_pythia1-4b' 'cpuep_ov_tinyllama' 'cpuep_torch_codeparrot-small'
 'cpuep_torch_phi2' 'cpuep_torch_pythia-410m' 'cpuep_torch_pythia1-4b'
 'cpuep_torch_tinyllama' 'cpuep_torchscript_codeparrot-small'
 'cpuep_torchscript_phi2' 'cpuep_torchscript_pythia-410m'
 'cpuep_torchscript_pythia1-4b' 'cpuep_torchscript_tinyllama'
 'gpuep_idle_idle' 'gpuep_onnx_codeparrot-small' 'gpuep_onnx_phi2'
 'gpuep_onnx_pythia-410m' 'gpuep_onnx_pythia1-4b' 'gpuep_onnx_tinyllama'
 'gpuep_torch_codeparrot-small' 'gpuep_torch_phi2'
 'gpuep_torch_pythia-410m' 'gpuep_torch_pythia1-4b'
 'gpuep_torch_tinyllama' 'gpuep_torchscript_codeparrot-small'
 'gpuep_torchscript_phi2' 'gpuep_torchscript_pythia-410m'
 'gpuep_torchscript_pythia1-4b' 'gpuep_torchscript_tinyllama']


,runtime,model,label,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config,config
0,idle,idle,idle_idle,0.130797,2.501843e+09,3.757879,cpuep_idle_idle
1,onnx,codeparrot-small,onnx_codeparrot-small,27.016334,9.507387e+09,14.280513,cpuep_onnx_codeparrot-small
2,onnx,phi2,onnx_phi2,79.619104,1.985527e+10,29.823487,cpuep_onnx_phi2
3,onnx,pythia-410m,onnx_pythia-410m,30.478362,1.173323e+10,17.623832,cpuep_onnx_pythia-410m
4,onnx,pythia1-4b,onnx_pythia1-4b,39.844363,9.955764e+09,14.953995,cpuep_onnx_pythia1-4b
5,onnx,tinyllama,onnx_tinyllama,81.593727,1.000236e+10,15.023981,cpuep_onnx_tinyllama
6,ov,codeparrot-small,ov_codeparrot-small,28.739977,1.167752e+10,17.540144,cpuep_ov_codeparrot-small
7,ov,phi2,ov_phi2,44.345280,1.285510e+10,19.308921,cpuep_ov_phi2
8,ov,pythia-410m,ov_pythia-410m,34.946671,1.179588e+10,17.717936,cpuep_ov_pythia-410m
9,ov,pythia1-4b,ov_pythia1-4b,41.236916,1.309087e+10,19.663062,cpuep_ov_pythia1-4b


In [174]:
print("CPUEP")
print(cpu_ep_data.describe())

print("CUDAEP")
print(gpu_ep_data.describe())


CPUEP
       avg_cpu_usage_config  avg_USED_MEMORY  avg_used_memory_pct_config
count             21.000000     2.100000e+01                   21.000000
mean              36.600883     1.187503e+10                   17.836809
std               18.928926     4.910566e+09                    7.375885
min                0.130797     2.501843e+09                    3.757879
25%               29.170391     9.507387e+09                   14.280513
50%               36.184416     1.167752e+10                   17.540144
75%               42.836422     1.309087e+10                   19.663062
max               81.593727     2.167293e+10                   32.553700
CUDAEP
       avg_cpu_usage_config  avg_USED_MEMORY  avg_used_memory_pct_config
count             16.000000     1.600000e+01                   16.000000
mean               2.100487     6.012612e+09                    9.031210
std                1.459955     2.470651e+09                    3.711028
min                0.110316     2.4952

In [175]:
global_df.describe()

,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config
count,37.000000,3.700000e+01,37.000000
mean,21.681793,9.339928e+09,14.028982
std,22.365066,4.960770e+09,7.451293
min,0.110316,2.495251e+09,3.747977
25%,2.573948,5.403162e+09,8.115786
50%,14.912778,9.375467e+09,14.082367
75%,38.703071,1.173323e+10,17.623832
max,81.593727,2.167293e+10,32.553700


In [176]:
df = global_df

def get_min_max_config(group):
    min_config = group.loc[group[dependent_variable].idxmin(), 'config']
    max_config = group.loc[group[dependent_variable].idxmax(), 'config']
    return pd.Series({'Min_Config': min_config, 'Max_Config': max_config})

# Applying the function to each group
result = df.groupby('model').apply(get_min_max_config)

# Output the result
result

,Min_Config,Max_Config
model,,
codeparrot-small,gpuep_torchscript_codeparrot-small,cpuep_torch_codeparrot-small
idle,gpuep_idle_idle,cpuep_idle_idle
phi2,gpuep_torchscript_phi2,cpuep_onnx_phi2
pythia-410m,gpuep_torchscript_pythia-410m,cpuep_torch_pythia-410m
pythia1-4b,gpuep_torchscript_pythia1-4b,cpuep_torch_pythia1-4b
tinyllama,gpuep_torchscript_tinyllama,cpuep_onnx_tinyllama


In [177]:
subset_cpu = global_df[global_df['config'].str.startswith('cpu', na=False)]
subset_gpu = global_df[global_df['config'].str.startswith('gpu', na=False)]

In [178]:
#result = subset_cpu.groupby('model').apply(get_min_max_config)
#result

In [179]:
#result = subset_gpu.groupby('model').apply(get_min_max_config)
#result

In [180]:
# Sort DataFrame by 'model' and 'dependent_variable' descending
sorted_df = subset_cpu.sort_values(by=['model',dependent_variable], ascending=[True, False])

# Group by model and select runtime_engines in order of descending dependent_variable
result = sorted_df.groupby('model')['runtime'].apply(list).reset_index()

# Display the result
print(f"Order {dependent_variable} by model: CPUExecutionProvider")
print(result)

Order avg_cpu_usage_config by model: CPUExecutionProvider
              model                         runtime
0  codeparrot-small  [torch, ov, onnx, torchscript]
1              idle                          [idle]
2              phi2  [onnx, ov, torchscript, torch]
3       pythia-410m  [torch, ov, onnx, torchscript]
4        pythia1-4b  [torch, ov, torchscript, onnx]
5         tinyllama  [onnx, ov, torchscript, torch]


In [181]:
# Sort DataFrame by 'model' and 'dependent_variable' descending
sorted_df = subset_gpu.sort_values(by=['model', dependent_variable], ascending=[True, False])

# Group by model and select runtime_engines in order of descending dependent_variable
result = sorted_df.groupby('model')['runtime'].apply(list).reset_index()

# Display the result
print("Order by model")
print(f"Order {dependent_variable} by model: GPUExecutionProvider")
print(result)

Order by model
Order avg_cpu_usage_config by model: GPUExecutionProvider
              model                     runtime
0  codeparrot-small  [onnx, torch, torchscript]
1              idle                      [idle]
2              phi2  [onnx, torch, torchscript]
3       pythia-410m  [onnx, torch, torchscript]
4        pythia1-4b  [onnx, torch, torchscript]
5         tinyllama  [onnx, torch, torchscript]


In [182]:
# all ordered by dependent variable

global_df.sort_values(by=dependent_variable, ascending=False)

,runtime,model,label,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config,config
5,onnx,tinyllama,onnx_tinyllama,81.593727,1.000236e+10,15.023981,cpuep_onnx_tinyllama
2,onnx,phi2,onnx_phi2,79.619104,1.985527e+10,29.823487,cpuep_onnx_phi2
7,ov,phi2,ov_phi2,44.345280,1.285510e+10,19.308921,cpuep_ov_phi2
14,torch,pythia1-4b,torch_pythia1-4b,44.169499,8.744414e+09,13.134493,cpuep_torch_pythia1-4b
10,ov,tinyllama,ov_tinyllama,42.997680,1.165492e+10,17.506201,cpuep_ov_tinyllama
13,torch,pythia-410m,torch_pythia-410m,42.836422,5.403162e+09,8.115786,cpuep_torch_pythia-410m
9,ov,pythia1-4b,ov_pythia1-4b,41.236916,1.309087e+10,19.663062,cpuep_ov_pythia1-4b
19,torchscript,pythia1-4b,torchscript_pythia1-4b,40.101432,1.797604e+10,27.000802,cpuep_torchscript_pythia1-4b
4,onnx,pythia1-4b,onnx_pythia1-4b,39.844363,9.955764e+09,14.953995,cpuep_onnx_pythia1-4b
11,torch,codeparrot-small,torch_codeparrot-small,38.703071,4.653973e+09,6.990471,cpuep_torch_codeparrot-small


In [183]:
def remove_condition_rows(condition_to_remove,df):
    # example: condition = merged_df['label']  == 'idle_idle'
    df1 = df.copy()
    return df1[~condition_to_remove]

In [184]:
condition = global_df['config'].str.endswith('idle')
global_df = remove_condition_rows( condition, global_df)
global_df.sort_values(by=dependent_variable, ascending=False)

,runtime,model,label,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config,config
5,onnx,tinyllama,onnx_tinyllama,81.593727,1.000236e+10,15.023981,cpuep_onnx_tinyllama
2,onnx,phi2,onnx_phi2,79.619104,1.985527e+10,29.823487,cpuep_onnx_phi2
7,ov,phi2,ov_phi2,44.345280,1.285510e+10,19.308921,cpuep_ov_phi2
14,torch,pythia1-4b,torch_pythia1-4b,44.169499,8.744414e+09,13.134493,cpuep_torch_pythia1-4b
10,ov,tinyllama,ov_tinyllama,42.997680,1.165492e+10,17.506201,cpuep_ov_tinyllama
13,torch,pythia-410m,torch_pythia-410m,42.836422,5.403162e+09,8.115786,cpuep_torch_pythia-410m
9,ov,pythia1-4b,ov_pythia1-4b,41.236916,1.309087e+10,19.663062,cpuep_ov_pythia1-4b
19,torchscript,pythia1-4b,torchscript_pythia1-4b,40.101432,1.797604e+10,27.000802,cpuep_torchscript_pythia1-4b
4,onnx,pythia1-4b,onnx_pythia1-4b,39.844363,9.955764e+09,14.953995,cpuep_onnx_pythia1-4b
11,torch,codeparrot-small,torch_codeparrot-small,38.703071,4.653973e+09,6.990471,cpuep_torch_codeparrot-small
